In [1]:
from lang import *
from transformers import LongformerModel, LongformerTokenizer

tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

In [2]:
lang_conf_params = {
    "max_len" : 4096
}
longformer_lang_conf = LangConf(vocab_size=50000,**lang_conf_params)
lang = LanguageIndex(config=longformer_lang_conf)
lang.load_tokenizer(tokenizer)

In [3]:
from datamodule import *

In [4]:
datamodule = DLNDDataModule(batch_size = 1)


In [5]:
datamodule.prepare_data(lang,None,combine=True)

Data Size  5352
train_samples:  4281


In [6]:

for i in datamodule.train_dataloader():
    print(i)
    break

[tensor([[  0, 250, 333,  ...,   1,   1,   1]]), tensor([0]), tensor([5058])]


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import LongformerModel


class Longformer_conf:
    encoder_dim = 768
    batch_size = None
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

class Lonformer_model(nn.Module):
    def __init__(self,conf):
        super(Lonformer_model,self).__init__()
        self.conf=conf
        self.encoder = LongformerModel.from_pretrained('allenai/longformer-base-4096')
        self.fc = nn.Linear(conf.encoder_dim,2)

    def forward(self,x0):
        enc = self.encoder.forward(x0)[0][:, 0, :]
        opt = self.fc(enc)
        opt = opt.unsqueeze(0)
        return opt




In [8]:
from novelty.train_utils import *


hparams = {
    "optim": "adam",
    "lr": 3e-4,
    "scheduler": "lambda",
    "weight_decay":0
    }

conf = Longformer_conf()
model = Novelty_longformer(Lonformer_model,conf,hparams)

In [9]:

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers import NeptuneLogger, TensorBoardLogger
from pytorch_lightning.metrics import Accuracy

tensorboard_logger = TensorBoardLogger("lightning_logs")
loggers = [tensorboard_logger]

EPOCHS=4

lr_logger = LearningRateLogger(logging_interval="step")
trainer = pl.Trainer(
    gpus=1,
    max_epochs=EPOCHS,
    progress_bar_refresh_rate=10,
    profiler=False,
    auto_lr_find=False,
    callbacks=[lr_logger],
    logger=loggers,
    row_log_interval=2,
    accumulate_grad_batches=4,
)
trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | Lonformer_model | 148 M 


Saving latest checkpoint..



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.5084),
 'test_f1': tensor(0.2542),
 'test_loss': tensor(0.6930, device='cuda:0'),
 'test_prec': tensor(0.2542),
 'test_recall': tensor(0.2542)}
--------------------------------------------------------------------------------



[{'test_acc': 0.5084112286567688,
  'test_f1': 0.2542056143283844,
  'test_loss': 0.6930384635925293,
  'test_prec': 0.2542056143283844,
  'test_recall': 0.2542056143283844}]